# Video "What is TensorFlow"

In [1]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.0.0-dev20190914


In [2]:
a = tf.constant(tf.ones((2,2)))
b = tf.Variable(tf.zeros((2,2)))

In [3]:
a

<tf.Tensor: id=2, shape=(2, 2), dtype=float32, numpy=
array([[1., 1.],
       [1., 1.]], dtype=float32)>

In [4]:
b

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[0., 0.],
       [0., 0.]], dtype=float32)>

In [5]:
b = tf.ones((2, 2))

In [6]:
c = a @ b
c

<tf.Tensor: id=18, shape=(2, 2), dtype=float32, numpy=
array([[2., 2.],
       [2., 2.]], dtype=float32)>

## First model in TensorFlow 2.0

In [7]:
x = tf.Variable(3,name = "x", trainable=True,dtype=tf.float32)
x

<tf.Variable 'x:0' shape=() dtype=float32, numpy=3.0>

In [8]:
x.numpy()

3.0

In [9]:
def y_fun():
    y = (x - 5) ** 2
    return y

optimizer = tf.keras.optimizers.SGD(learning_rate=0.3)
for i in range(10):
    step = optimizer.minimize(y_fun, var_list=[x])
    print(x.numpy(),y_fun().numpy())

4.2 0.6400003
4.68 0.10240011
4.8719997 0.016384067
4.9488 0.0026214311
4.97952 0.00041943678
4.991808 6.710989e-05
4.996723 1.0737582e-05
4.998689 1.7182631e-06
4.9994755 2.7512215e-07
4.99979 4.4019544e-08


### Simple optimization (with tf.Print)

In [10]:
x = tf.Variable(3,name = "x", trainable=True,dtype=tf.float32)

def y_fun():
    y = (x - 5) ** 2
    #tf.print(y, x,"x, f:")
    tf.print("x : ", x,", y: " , y)
    return y

optimizer = tf.keras.optimizers.SGD(learning_rate=0.3)
for i in range(10):
    step = optimizer.minimize(y_fun, var_list=[x])

x :  3 , y:  4
x :  4.2 , y:  0.640000284
x :  4.68 , y:  0.102400109
x :  4.87199974 , y:  0.016384067
x :  4.9488 , y:  0.00262143114
x :  4.97952 , y:  0.000419436779
x :  4.99180794 , y:  6.71098896e-05
x :  4.99672318 , y:  1.07375818e-05
x :  4.99868917 , y:  1.71826309e-06
x :  4.99947548 , y:  2.75122147e-07


### Simple optimization (with TensorBoard logging)

In [11]:
# ! pip install tf-nightly-2.0-preview tf_estimator_nightly

In [12]:
%load_ext tensorboard

In [13]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

In [14]:
directory = 'logs/simple1'
summary_writer = tf.summary.create_file_writer(directory)

In [15]:
x = tf.Variable(3,name = "x", trainable=True,dtype=tf.float32)

def y_fun():
    y = (x - 5) ** 2
    #tf.print(y, x,"x, f:")
    #tf.print("x : ", x," y: " , y)
    return y

optimizer = tf.keras.optimizers.SGD(learning_rate=0.3)
for i in range(10):
    step = optimizer.minimize(y_fun, var_list=[x])
    
    with summary_writer.as_default():
        tf.summary.scalar('x', x, step=i)
        tf.summary.scalar('y', y_fun(), step=i)

In [16]:
%tensorboard --logdir logs/simple1

### Training a linear model

In [17]:
# generate model data
N = 1000
D = 3
x = np.random.random((N, D))
w = np.random.random((D, 1))
y = x @ w + np.random.randn(N, 1) * 0.20

print(x.shape, y.shape)
print(w.T)

(1000, 3) (1000, 1)
[[0.30736706 0.11011295 0.55950296]]


In [18]:
weights = tf.Variable(np.ones((D, 1)),name = "w", trainable=True,dtype=tf.float32)

def loss_function():
    pred = x @ weights
    loss = tf.reduce_mean((y - pred) ** 2)
    return loss

In [19]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

In [20]:
for i in range(300):
    step = optimizer.minimize(loss_function, var_list=[weights])
    if i % 50 == 0:
        print(loss_function().numpy())

0.85446465
0.044633225
0.04343107
0.04319504
0.043148473
0.04313925


In [21]:
# found weights
weights

<tf.Variable 'w:0' shape=(3, 1) dtype=float32, numpy=
array([[0.3166873 ],
       [0.09941876],
       [0.5571117 ]], dtype=float32)>

In [22]:
# true weights
w

array([[0.30736706],
       [0.11011295],
       [0.55950296]])

### Another type of execution

In [23]:
N,D

(1000, 3)

In [24]:
import tensorflow as tf
import numpy as np

#x = np.array([
#    [100, 105.4, 108.3, 111.1, 113, 114.7],
#    [11, 11.8, 12.3, 12.8, 13.1, 13.6],
#    [55, 56.3, 57, 58, 59.5, 60.4]
#])

#y = np.array([4000, 4200.34, 4700, 5300, 5800, 6400])

x = np.random.random((N, D))
w = np.random.random((D, 1))
b = np.random.randn(1, 1) * 0.20

y = x @ w + b

In [25]:
class Model(object):
    def __init__(self, x, y):
        
        self.W = tf.Variable(tf.random.normal((D, 1)))
        self.b = tf.Variable(tf.random.normal((1, 1)))

    def __call__(self, x):
        return x @ self.W + self.b

In [26]:
def loss(predicted_y, desired_y):
    return tf.reduce_sum(tf.square(predicted_y - desired_y))

In [27]:
optimizer = tf.optimizers.Adam(0.1)

def train(model, inputs, outputs):
    with tf.GradientTape() as t:
        current_loss = loss(model(inputs), outputs)
    grads = t.gradient(current_loss, [model.W, model.b])
    optimizer.apply_gradients(zip(grads,[model.W, model.b]))
    print(current_loss.numpy())

In [28]:
model = Model(x, y)

for i in range(100):
    train(model,x,y)

3589.9639
2718.1519
1981.5405
1380.9463
914.73737
577.9038
361.10626
249.89703
224.56374
261.09662
333.49518
417.01352
491.3374
542.65234
564.1449
555.17993
519.7427
464.69894
398.20996
328.44724
262.63
206.36424
163.2549
134.77457
120.37453
117.8186
123.6959
134.0427
144.9779
153.2528
156.63461
154.08
145.70038
132.55618
116.33998
99.01486
82.46812
68.22835
57.27758
49.97361
46.08072
44.891525
45.410194
46.559525
47.37393
47.14648
45.510025
42.44686
38.235268
33.351414
28.350632
23.75213
19.947523
17.146275
15.362936
14.442886
14.116438
14.067074
13.99882
13.689567
13.021869
11.988346
10.674231
9.223955
7.8009276
6.549668
5.5677624
4.891989
4.499402
4.3209543
4.262887
4.230162
4.1465487
3.9675183
3.6841807
3.318757
2.9139042
2.5191905
2.1782117
1.9191679
1.7505054
1.6618452
1.6291587
1.6223223
1.6129236
1.580427
1.5154893
1.4200935
1.3049538
1.1852534
1.0760139
0.9882766
0.9269113
0.89032686
0.8718538
0.862182
0.85207695
0.83463776
0.8065926
0.76846504


In [29]:
model.W.numpy()

array([[0.42773825],
       [0.3165277 ],
       [0.2242432 ]], dtype=float32)

In [30]:
model.b.numpy()

array([[-0.06439339]], dtype=float32)

In [31]:
w

array([[0.47887999],
       [0.36044085],
       [0.27508401]])

In [32]:
b

array([[-0.14993601]])